In [ ]:
!pip install pyttsx3 SpeechRecognition requests google-api-python-client googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=fe94883e30cd3e43839cc7ff8b9b3c33e624dd936a5ddef80b29acd3c585e1ea
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-17' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3966036785.py", li

In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok scikit-learn pandas numpy -q
from pyngrok import ngrok
# optional: kill any old tunnels (defensive)
ngrok.kill()


In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok scikit-learn pandas numpy pymupdf -q

# ============================================================
# 🩺 SANA – FINAL DEMO VERSION (OPTIONAL UPLOAD)
# ============================================================
import nest_asyncio, threading, uvicorn, random, re
from datetime import datetime
from pyngrok import ngrok
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse, JSONResponse
import pandas as pd, numpy as np, fitz
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# -----------------------------
# 1️⃣ Load data & train model
# -----------------------------
train_df = pd.read_csv("/content/dataset.csv")
sev_df   = pd.read_csv("/content/Symptom-severity.csv")
desc_df  = pd.read_csv("/content/symptom_Description.csv")
prec_df  = pd.read_csv("/content/symptom_precaution.csv")

LABEL = "Disease"
symptom_cols = [c for c in train_df.columns if c.startswith("Symptom_")]

# all unique symptoms
all_symptoms = sorted({
    str(s).strip().replace(" ", "_")
    for c in symptom_cols
    for s in train_df[c].dropna().astype(str)
})

X = pd.DataFrame(0, index=train_df.index, columns=all_symptoms)
for i, row in train_df.iterrows():
    for c in symptom_cols:
        s = str(row[c]).strip().replace(" ", "_")
        if s in all_symptoms:
            X.at[i, s] = 1

y = train_df[LABEL]
le = LabelEncoder()
y_enc = le.fit_transform(y)

clf = RandomForestClassifier(n_estimators=120, random_state=42)
clf.fit(X, y_enc)

# store disease symptom sets
disease_symptoms = {}
for d in le.classes_:
    vals = (
        train_df.loc[train_df[LABEL] == d, symptom_cols]
        .melt()["value"].dropna().astype(str).str.replace(" ", "_")
    )
    disease_symptoms[d] = set(vals)

# -----------------------------
# 2️⃣ Helper functions
# -----------------------------
def severity_from_symptoms(symptoms, age):
    m = sev_df[sev_df["Symptom"].isin(symptoms)]["weight"].mean()
    if np.isnan(m): sev = "Unknown"
    elif m > 4:     sev = "High"
    elif m > 2:     sev = "Moderate"
    else:           sev = "Low"
    if sev != "Unknown":
        if age > 60: sev = "High"
        elif age < 12 and sev == "Moderate": sev = "High"
    return sev

def severity_from_text(txt):
    t = txt.lower()
    if any(k in t for k in ["critical","emergency","severe","icu","pneumonia","sepsis"]):
        return "High"
    if any(k in t for k in ["infection","positive","antibiotic","inflammation","abnormal"]):
        return "Moderate"
    return "Low"

def combine_severity(model_sev, report_sev):
    order = {"Unknown": 0, "Low": 1, "Moderate": 2, "High": 3}
    rev   = {v:k for k,v in order.items()}
    return rev[max(order.get(model_sev,0), order.get(report_sev,0))]

LAST_REPORT = {"severity_hint": "Unknown", "preview": ""}

# -----------------------------
# 3️⃣ FastAPI app
# -----------------------------
app = FastAPI(title="Sana – Optional Upload Demo")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# -------- HTML UI --------
@app.get("/", response_class=HTMLResponse)
def ui():
    return """
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Sana – Health Chatbot (Demo)</title>
<style>
body{font-family:Arial;background:#0b1020;color:white;text-align:center;padding:30px;}
h1{color:#00bcd4;}
.card{background:#111827;padding:20px;border-radius:16px;display:inline-block;text-align:left;width:420px;}
input,button{padding:10px;border-radius:8px;margin:6px;border:none;width:95%;}
button{background:#00bcd4;color:#000;font-weight:bold;cursor:pointer;}
.muted{color:#aaa;font-size:13px;}
</style>
</head>
<body>
<h1>🩺 Sana Chatbot</h1>
<div class="card">
  <label>Name:</label><br><input id="name" placeholder="Your name"><br>
  <label>Age:</label><br><input id="age" type="number" placeholder="25"><br>
  <label>City:</label><br><input id="city" placeholder="Hyderabad"><br>
  <label>Symptoms (comma separated):</label><br>
  <input id="symptoms" placeholder="fever, cough, sore throat"><br>
  <button id="mic">🎙️ Voice Input</button>
  <p id="mic-status" class="muted">You can speak symptoms aloud</p>
  <hr>
  <label>Upload medical report (PDF) – optional:</label><br>
  <input id="report" type="file" accept=".pdf"><br>
  <button id="upload">Upload</button>
  <p id="upload-msg" class="muted">(You can skip this step)</p>
  <button id="predict">Predict</button>
  <div id="result"></div>
</div>

<script>
let reportHint="Unknown";

async function uploadReport(){
  const file=document.getElementById('report').files[0];
  if(!file){alert('No file selected');return;}
  document.getElementById('upload-msg').textContent='Analyzing report...';
  const fd=new FormData(); fd.append('file',file);
  const res=await fetch('/upload_report',{method:'POST',body:fd});
  const data=await res.json();
  reportHint=data.severity_hint;
  document.getElementById('upload-msg').textContent='Report analyzed. Severity hint: '+reportHint;
}

async function predict(){
  const name=document.getElementById('name').value;
  const age=parseInt(document.getElementById('age').value||'0');
  const city=document.getElementById('city').value;
  const symptoms=document.getElementById('symptoms').value.split(',').map(s=>s.trim()).filter(Boolean);
  if(!name||!age||!city||symptoms.length==0){alert('Fill all required fields');return;}
  document.getElementById('result').innerHTML='⏳ Analyzing...';
  const res=await fetch('/predict',{
    method:'POST',headers:{'Content-Type':'application/json'},
    body:JSON.stringify({name,age,city,symptoms})
  });
  const r=await res.json();
  let html=`<hr><b>Disease:</b> ${r.disease}<br><b>Severity:</b> ${r.severity}<br><b>Description:</b> ${r.desc}<br><b>Precautions:</b> ${(r.precautions||[]).join(', ')}`;
  html+=`<br><br><b>Nearby Doctors:</b><ul>`+r.doctors.map(d=>`<li>${d.name} ⭐${d.rating} (${d.experience} yrs)</li>`).join('')+`</ul>`;
  document.getElementById('result').innerHTML=html;
}

document.getElementById('upload').onclick=uploadReport;
document.getElementById('predict').onclick=predict;

// voice input
const SR=window.SpeechRecognition||window.webkitSpeechRecognition;
if(SR){
  const rec=new SR(); rec.lang='en-US'; rec.interimResults=false;
  rec.onresult=e=>{
    const t=e.results[0][0].transcript;
    document.getElementById('symptoms').value=t.toLowerCase();
    document.getElementById('mic-status').textContent='Heard: '+t;
  };
  rec.onend=()=>{document.getElementById('mic').textContent='🎙️ Voice Input';};
  document.getElementById('mic').onclick=()=>{
    rec.start(); document.getElementById('mic').textContent='🛑 Stop';};
}else{
  document.getElementById('mic').disabled=true;
  document.getElementById('mic-status').textContent='Speech not supported in this browser.';
}
</script>
</body>
</html>
"""

# ---------- UPLOAD (optional) ----------
@app.post("/upload_report")
async def upload_report(file: UploadFile = File(...)):
    try:
        if not file.filename.lower().endswith(".pdf"):
            return JSONResponse({"severity_hint": "Unknown", "preview": "(Only PDF supported)"})
        path = f"/tmp/{file.filename}"
        with open(path, "wb") as f: f.write(await file.read())
        txt = ""
        doc = fitz.open(path)
        for p in doc: txt += p.get_text()
        doc.close()
        hint = severity_from_text(txt)
        preview = re.sub(r"\s+", " ", txt.strip())[:200]
        LAST_REPORT.update({"severity_hint": hint, "preview": preview})
        return {"severity_hint": hint, "preview": preview}
    except Exception as e:
        return JSONResponse({"severity_hint": "Unknown", "preview": f"Error: {e}"})

# ---------- PREDICT ----------
@app.post("/predict")
def predict(payload: dict):
    name = payload.get("name","")
    age  = int(payload.get("age",0))
    city = payload.get("city","")
    symptoms = [s.strip().replace(" ","_") for s in payload.get("symptoms",[]) if s.strip()]

    row = {col:1 if col in symptoms else 0 for col in all_symptoms}
    Xq = pd.DataFrame([row])
    probs = clf.predict_proba(Xq)[0]
    labels = le.inverse_transform(np.arange(len(probs)))
    df = pd.DataFrame({"Disease":labels,"Prob":probs})
    df["Overlap"]=[len(disease_symptoms[d].intersection(symptoms)) for d in df["Disease"]]
    if df["Overlap"].max()==0: df["Score"]=df["Prob"]
    else: df["Score"]=df["Prob"]*0.7+(df["Overlap"]/df["Overlap"].max())*0.3
    disease = df.sort_values("Score",ascending=False).iloc[0]["Disease"]

    desc = desc_df.loc[desc_df["Disease"]==disease,"Description"].values
    desc = desc[0] if len(desc) else "Description not available."
    prec = prec_df.loc[prec_df["Disease"]==disease]
    precautions = [p for p in prec.values.flatten() if isinstance(p,str) and p!=disease]

    model_sev = severity_from_symptoms(symptoms, age)
    report_sev = LAST_REPORT.get("severity_hint","Unknown")
    final_sev = combine_severity(model_sev, report_sev)

    # demo doctors
    docs=[]
    for i in range(3):
        docs.append({
            "name":f"Dr. {random.choice(['A','B','C','D','E','F','G','H','I','J'])}. {random.choice(['Rao','Patel','Singh','Menon','Iyer'])}",
            "rating":round(random.uniform(4.0,5.0),1),
            "experience":random.randint(5,25)
        })

    return {"name":name,"age":age,"city":city,"disease":disease,"severity":final_sev,"desc":desc,"precautions":precautions,"doctors":docs}

# -----------------------------
# 4️⃣ Run FastAPI + ngrok
# -----------------------------
nest_asyncio.apply()
ngrok.kill()

def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=start_server, daemon=True)
thread.start()

public_url = ngrok.connect(8000).public_url
print(f"\n🚀 SANA Demo running at: {public_url}\n")
print("ℹ️ Upload PDF is optional; you can predict without it.")


INFO:     Started server process [1174]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



🚀 SANA Demo running at: https://rogelio-pyramidlike-epidemically.ngrok-free.dev

ℹ️ Upload PDF is optional; you can predict without it.


In [ ]:
import os
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with the key you just provided or obtained from your ngrok dashboard.
# You can also set this as an environment variable or using `userdata.set` in Colab for better security.
# For this demonstration, we'll set it directly.

NGROK_AUTH_TOKEN = '35IaHXYO9zahrFC3iprixHqjXZv_2FqSsGJEhMttfbRCsKRGw'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("ngrok authentication token set.")
print("Now you can re-run the cell that starts the FastAPI app and ngrok tunnel.")

ngrok authentication token set.
Now you can re-run the cell that starts the FastAPI app and ngrok tunnel.


In [ ]:
# --- Reset Sana environment (run this before your main cell) ---
from pyngrok import ngrok
import psutil, signal

# Kill any process still using port 8000
for conn in psutil.net_connections():
    if conn.laddr.port == 8000 and conn.pid:
        try:
            psutil.Process(conn.pid).send_signal(signal.SIGTERM)
        except Exception:
            pass

# Close any leftover ngrok tunnels
ngrok.kill()

print("✅ Cleared old servers and tunnels. You can now re-run the Sana cell safely.")


✅ Cleared old servers and tunnels. You can now re-run the Sana cell safely.
